<a href="https://colab.research.google.com/github/levinaanora/Corona-Virus-Tweet-NLP/blob/main/Coronavirus_tweets_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [68]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [69]:
data = pd.read_csv("Corona_NLP_train.csv", header = 0)
data_test = pd.read_csv("Corona_NLP_test.csv", header = 0)
data.tail()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
41152,44951,89903,"Wellington City, New Zealand",14-04-2020,Airline pilots offering to stock supermarket s...,Neutral
41153,44952,89904,NaN,14-04-2020,Response to complaint not provided citing COVI...,Extremely Negative
41154,44953,89905,NaN,14-04-2020,You know itÂ’s getting tough when @KameronWild...,Positive
41155,44954,89906,NaN,14-04-2020,Is it wrong that the smell of hand sanitizer i...,Neutral
41156,44955,89907,i love you so much || he/him,14-04-2020,@TartiiCat Well new/used Rift S are going for ...,Negative


In [70]:
data_test.tail()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
3793,3794,48746,Israel ??,16-03-2020,Meanwhile In A Supermarket in Israel -- People...,Positive
3794,3795,48747,"Farmington, NM",16-03-2020,Did you panic buy a lot of non-perishable item...,Negative
3795,3796,48748,"Haverford, PA",16-03-2020,Asst Prof of Economics @cconces was on @NBCPhi...,Neutral
3796,3797,48749,NaN,16-03-2020,Gov need to do somethings instead of biar je r...,Extremely Negative
3797,3798,48750,"Arlington, Virginia",16-03-2020,I and @ForestandPaper members are committed to...,Extremely Positive


In [71]:
del data["UserName"], data["ScreenName"], data["Location"], data["TweetAt"]
data.head()

,OriginalTweet,Sentiment
0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,advice Talk to your neighbours family to excha...,Positive
2,Coronavirus Australia: Woolworths to give elde...,Positive
3,My food stock is not the only one which is emp...,Positive
4,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [72]:
del data_test["UserName"], data_test["ScreenName"], data_test["Location"], data_test["TweetAt"]
data_test.head()

,OriginalTweet,Sentiment
0,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative
1,When I couldn't find hand sanitizer at Fred Me...,Positive
2,Find out how you can protect yourself and love...,Extremely Positive
3,#Panic buying hits #NewYork City as anxious sh...,Negative
4,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral


In [73]:
data["Sentiment"].value_counts()

Positive              11422
Negative               9917
Neutral                7713
Extremely Positive     6624
Extremely Negative     5481
Name: Sentiment, dtype: int64

In [74]:
data_test["Sentiment"].value_counts()

Negative              1041
Positive               947
Neutral                619
Extremely Positive     599
Extremely Negative     592
Name: Sentiment, dtype: int64

In [75]:
missing_data = pd.DataFrame({'total_missing': data.isnull().sum(), 'perc_missing': (data.isnull().sum()/41156)*100})
missing_data

,total_missing,perc_missing
OriginalTweet,0,0.0
Sentiment,0,0.0


In [76]:
max_tweet = 0
tweet_length = []
for tweet in data["OriginalTweet"]:
  tweet_length.append(len(tweet))
  if len(tweet) > max_tweet:
    max_tweet = len(tweet)

print("The longest tweet is: {}".format(max_tweet))

The longest tweet is: 355


In [77]:
def to_3_classes(x):
  if x=="Extremely Negative":
    return "Negative"
  elif x=="Extremely Positive":
    return "Positive"
  else:
    return x

data["Sentiment"] = data["Sentiment"].apply(to_3_classes)
data_test["Sentiment"] = data_test["Sentiment"].apply(to_3_classes)

In [78]:
data["Sentiment"].value_counts()

Positive    18046
Negative    15398
Neutral      7713
Name: Sentiment, dtype: int64

In [79]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
data["Sentiment"] = label_encoder.fit_transform(data["Sentiment"])
data["Sentiment"]

0        1
1        2
2        2
3        2
4        0
        ..
41152    1
41153    0
41154    2
41155    1
41156    0
Name: Sentiment, Length: 41157, dtype: int64

In [80]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
X = data["OriginalTweet"]
y = data["Sentiment"]

X_train , X_test , y_train , y_test = train_test_split(X, y, test_size=0.2, random_state=0)
y_train = to_categorical(y_train, 3)
y_test  = to_categorical(y_test, 3)

In [14]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer(num_words = 20000, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)
list_tokenized_train = tokenizer.texts_to_sequences(X_train)
list_tokenized_test = tokenizer.texts_to_sequences(X_test)
embed_size = 128 
padded_train = pad_sequences(list_tokenized_train, maxlen=max_tweet)
padded_test = pad_sequences(list_tokenized_test, maxlen=max_tweet)

In [18]:
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Bidirectional
from tensorflow.keras.layers import GlobalMaxPool1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
model = Sequential()
model.add(Embedding(20000, embed_size))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(3, activation='softmax'))
adam = Adam(learning_rate=0.01)

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 128)         2560000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200)               183200    
_________________________________________________________________
dense_4 (Dense)              (None, 64)                12864     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 195       
Total params: 2,756,259
Trainable params: 2,756,259
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])

In [20]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(patience=5, monitor='val_accuracy', restore_best_weights=True)
model.fit(padded_train,y_train, epochs=20,validation_data=(padded_test,y_test), callbacks=[es])

Epoch 1/20
1029/1029 [==============================] - 134s 127ms/step - loss: 0.7928 - accuracy: 0.6453 - val_loss: 0.6008 - val_accuracy: 0.7766
Epoch 2/20
1029/1029 [==============================] - 129s 125ms/step - loss: 0.5273 - accuracy: 0.8147 - val_loss: 0.6551 - val_accuracy: 0.7515
Epoch 3/20
1029/1029 [==============================] - 128s 124ms/step - loss: 0.4221 - accuracy: 0.8600 - val_loss: 0.5049 - val_accuracy: 0.8313
Epoch 4/20
1029/1029 [==============================] - 128s 124ms/step - loss: 0.3504 - accuracy: 0.8870 - val_loss: 0.5400 - val_accuracy: 0.8147
Epoch 5/20
1029/1029 [==============================] - 128s 124ms/step - loss: 0.3179 - accuracy: 0.8994 - val_loss: 0.5508 - val_accuracy: 0.8189
Epoch 6/20
1029/1029 [==============================] - 128s 124ms/step - loss: 0.3003 - accuracy: 0.9052 - val_loss: 0.5375 - val_accuracy: 0.8246
Epoch 7/20
1029/1029 [==============================] - 127s 124ms/step - loss: 0.2792 - accuracy: 0.9122 - val_

In [ ]:
model.evaluate(padded_test,y_test)

In [22]:
model.save('corona_nlp_ver2.h5')